## Import libraries

In [1]:
import ee
import geemap

## Create an interactive map

In [2]:
Map = geemap.Map(center=[40, -100], zoom=4)

## Add Earth Engine Python script

In [3]:
# Add Earth Engine dataset
image = ee.Image("USGS/SRTMGL1_003")

#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#  Chapter:      F5.3 Advanced Vector Operations
#  Checkpoint:   F53a
#  Author:       Ujaval Gandhi
#  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

blocks = ee.FeatureCollection('TIGER/2010/Blocks')
roads = ee.FeatureCollection('TIGER/2016/Roads')
sfNeighborhoods = ee.FeatureCollection(
'projects/gee-book/assets/F5-0/SFneighborhoods')

geometry = sfNeighborhoods.geometry()
Map.centerObject(geometry)

# Creating a Choropleth Map

# Filter blocks to the San Francisco boundary.
sfBlocks = blocks.filter(ee.Filter.bounds(geometry))

# Visualize with a single color.
Map.addLayer(sfBlocks, {
    'color': '#de2d26'
}, 'Census Blocks (single color)')

# Visualize with values in a column using paint().

# Add a pop_density column.

def func_uzj(f):
    # Get the polygon area in square miles.
    area_sqmi = f.area().divide(2.59e6)
    population = f.get('pop10')
    # Calculate population density.
    density = ee.Number(population).divide(area_sqmi)
    return f.set({
        'area_sqmi': area_sqmi,
        'pop_density': density
        })

sfBlocks = sfBlocks.map(func_uzj)











# Calculate the statistics of the newly computed column.
stats = sfBlocks.aggregate_stats('pop_density')
print(stats.getInfo())

# Create an empty image into which to paint the features.
# Cast to 32-bit integer which supports storing values
# up to 2,147,483,647.

empty = ee.Image().int32()

# use paint() to color image with the values from the
# 'pop_density' column.
sfBlocksPaint = empty.paint(
featureCollection = sfBlocks,
color = 'pop_density',
)

palette = ['fee5d9', 'fcae91', 'fb6a4a', 'de2d26', 'a50f15']
visParams = {
    'min': 0,
    'max': 50000,
    'palette': palette
}
Map.addLayer(sfBlocksPaint.clip(geometry), visParams,
'Population Density')

# Filter roads to San Francisco boundary.
sfRoads = roads.filter(ee.Filter.bounds(geometry))

Map.addLayer(sfRoads, {
    'color': 'blue'
}, 'Roads (default)')

# Visualize with draw().
sfRoadsDraw = sfRoads.draw(
color = 'blue',
strokeWidth = 1
)
Map.addLayer(sfRoadsDraw, {}, 'Roads (Draw)')

styles = ee.Dictionary({
'S1100': {
    'color': 'blue',
    'width': 3
},
'S1200': {
    'color': 'green',
    'width': 2
},
'S1400': {
    'color': 'orange',
    'width': 1
}
})
defaultStyle = {
    'color': 'gray',
    'width': 1
}


def func_fqu(f):
    classcode = f.get('mtfcc')
    style = styles.get(classcode, defaultStyle)
    return f.set('style', style)

sfRoads = sfRoads.map(func_fqu)





sfRoadsStyle = sfRoads.style(
styleProperty = 'style'
)
Map.addLayer(sfRoadsStyle.clip(geometry), {}, 'Roads (Style)')

#  -----------------------------------------------------------------------
#  CHECKPOINT
#  -----------------------------------------------------------------------

{'max': 560799.3972602487, 'mean': 19815.271612013155, 'min': 0, 'sample_sd': 24428.737718087283, 'sample_var': 596763226.4991002, 'sum': 146434857.21277723, 'sum_sq': 7311129949739.396, 'total_count': 7390, 'total_sd': 24427.084838237082, 'total_var': 596682473.6944319, 'valid_count': 7390, 'weight_sum': 7390, 'weighted_sum': 146434857.21277723}


## Display the interactive map

In [4]:
Map

Map(center=[37.770715116445906, -122.4359566847391], controls=(WidgetControl(options=['position', 'transparent…